In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_credit = pd.read_csv("/kaggle/input/german-credit-data-with-risk/german_credit_data.csv", index_col=0)

In [ ]:
df_credit.head()

In [ ]:
g = []
for i in df_credit['Checking account']:
    if i == "moderate":
        g.append(np.random.randint(6000,25000))
    elif i == "little":
        g.append(np.random.randint(0,6000))
    elif i == "rich":
        g.append(np.random.randint(25000,400000))
    else:
        g.append(np.random.randint(6000,25000))
df_credit['Checking account'] = g

In [ ]:
g = []
for i in df_credit['Saving accounts']:
    if i == "moderate":
        g.append(np.random.randint(6000,25000))
    elif i == "little":
        g.append(np.random.randint(0,6000))
    elif i == "quite rich":
        g.append(np.random.randint(25000,150000))
    elif i == "rich":
        g.append(np.random.randint(150000,400000))
    else:
        g.append(np.random.randint(6000,25000))
df_credit['Saving accounts'] = g

In [ ]:
df_credit["Annual income"] = (df_credit['Saving accounts'] + df_credit['Checking account'])*4

In [ ]:
df_credit['People employed'] = np.random.randint(1,20, size=len(df_credit))
g = []
for i in df_credit['People employed']:
    g.append(np.random.randint(1,i+2))
df_credit['Skilled personnel'] = g

In [ ]:
df_credit["Years running"] = np.random.randint(1,50, size=len(df_credit))
df_credit["Customer facing"] = np.random.randint(0,2, size=len(df_credit))
df_credit["Place of operation owned"] = np.random.randint(0,3, size=len(df_credit))
df_credit["Cash transactions"] = np.random.randint(0,2, size=len(df_credit))
df_credit["Value of assets"] = df_credit['Annual income']*(df_credit['Years running']+1)+df_credit['Checking account']

In [ ]:
g = [0]*len(df_credit)
for i,row in df_credit.iterrows():
    if (row["Value of assets"]>500000)&(row["Years running"]>20):
        g[i] = np.random.choice([0,1], p=[0.3,.7])
    if (row['Customer facing']==1)&(row["Place of operation owned"]==0):
        g[i] = np.random.choice([0,1], p=[0.3,.7])
    if (row['Checking account']>1000000)&(row["Years running"]>20):
        g[i] = np.random.choice([0,1], p=[0.3,.7])
    if (row['People employed']>15)&(row["Skilled personnel"]>5):
        g[i] = np.random.choice([0,1], p=[0.3,.7])
    if (row['Credit amount'] >10000) :
        g[i] = np.random.choice([0,1], p=[0.7,.3])
    if (row['Years running']<5) & (row['Customer facing']==0):
        g[i] = np.random.choice([0,1], p=[0.7,.3])
df_credit['Risk'] = g

In [ ]:
sns.distplot(df_credit['Credit amount'])

In [ ]:
# sns.distplot(df_credit['Risk']/1000)

In [ ]:
# df_credit['Risk'] = df_credit['Risk'].apply(lambda x : x/max(df_credit['Risk']))

In [ ]:
sns.distplot(df_credit["Value of assets"])

In [ ]:
del df_credit['Saving accounts']
del df_credit['Purpose']
del df_credit['Housing']
del df_credit['Sex']

In [ ]:
X = df_credit.drop('Risk', 1)
X = X.values
y = df_credit["Risk"].values

In [ ]:
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = XGBClassifier(random_state=2, verbosity=1)
model.fit(X_train, y_train)

In [ ]:
sns.countplot(model.predict(X_test))

In [ ]:
model.predict_proba(X_test)

In [ ]:
def plot_feature_importances(clf, X_train, y_train=None, 
                             top_n=10, figsize=(8,8), print_table=False, title="Feature Importances"):
    '''
    plot feature importances of a tree-based sklearn estimator
    
    Note: X_train and y_train are pandas DataFrames
    
    Note: Scikit-plot is a lovely package but I sometimes have issues
              1. flexibility/extendibility
              2. complicated models/datasets
          But for many situations Scikit-plot is the way to go
          see https://scikit-plot.readthedocs.io/en/latest/Quickstart.html
    
    Parameters
    ----------
        clf         (sklearn estimator) if not fitted, this routine will fit it
        
        X_train     (pandas DataFrame)
        
        y_train     (pandas DataFrame)  optional
                                        required only if clf has not already been fitted 
        
        top_n       (int)               Plot the top_n most-important features
                                        Default: 10
                                        
        figsize     ((int,int))         The physical size of the plot
                                        Default: (8,8)
        
        print_table (boolean)           If True, print out the table of feature importances
                                        Default: False
        
    Returns
    -------
        the pandas dataframe with the features and their importance
        
    Author
    ------
        George Fisher
    '''
    
    __name__ = "plot_feature_importances"
    
    import pandas as pd
    import numpy  as np
    import matplotlib.pyplot as plt
    
    from xgboost.core     import XGBoostError
    from lightgbm.sklearn import LightGBMError
    
    try: 
        if not hasattr(clf, 'feature_importances_'):
            clf.fit(X_train.values, y_train.values.ravel())

            if not hasattr(clf, 'feature_importances_'):
                raise AttributeError("{} does not have feature_importances_ attribute".
                                    format(clf.__class__.__name__))
                
    except (XGBoostError, LightGBMError, ValueError):
        clf.fit(X_train.values, y_train.values.ravel())
            
    feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
    feat_imp['feature'] = X_train.columns
    feat_imp.sort_values(by='importance', ascending=False, inplace=True)
    feat_imp = feat_imp.iloc[:top_n]
    
    feat_imp.sort_values(by='importance', inplace=True)
    feat_imp = feat_imp.set_index('feature', drop=True)
    feat_imp.plot.barh(title=title, figsize=figsize)
    plt.xlabel('Feature Importance Score')
    plt.show()
    
    if print_table:
        from IPython.display import display
        print("Top {} features in descending order of importance".format(top_n))
        display(feat_imp.sort_values(by='importance', ascending=False))
        
    return feat_imp

In [ ]:
from xgboost              import XGBClassifier
from sklearn.ensemble     import ExtraTreesClassifier
from sklearn.tree         import ExtraTreeClassifier
from sklearn.tree         import DecisionTreeClassifier
from sklearn.ensemble     import GradientBoostingClassifier
from sklearn.ensemble     import BaggingClassifier
from sklearn.ensemble     import AdaBoostClassifier
from sklearn.ensemble     import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm             import LGBMClassifier


clfs = [XGBClassifier(),              LGBMClassifier(), 
        ExtraTreesClassifier(),       ExtraTreeClassifier(),
        BaggingClassifier(),          DecisionTreeClassifier(),
        GradientBoostingClassifier(), LogisticRegression(),
        AdaBoostClassifier(),         RandomForestClassifier()]

for clf in clfs:
    try:
        _ = plot_feature_importances(clf, df_credit.drop(['Risk'],1), df_credit["Risk"], top_n=df_credit.drop(['Risk'],1).shape[1], title=clf.__class__.__name__)
    except AttributeError as e:
        print(e)

In [ ]:
model.predict_proba(X_test)

In [ ]:
df_credit.head()

In [ ]:
df_credit.to_csv("biz_loan_risk.csv", index= False)